# Personalized training: StyleGan2-ADA

In this notebook, we will perform transfer learning with StyleGAN2 and custom datasets.

This means that we will not train the GAN network on our images from scratch (as it takes around two weeks) but we will use the model already trained on the other images as a starting point. It will reduce the training time to about 10 hours by skipping the early stages where the neural network learns low-level features of images that are almost the same for any type of images.

In [ ]:
#@title 1. Mount Google Drive
#@markdown Access your Google Drive to load images from databases or pre-trained models and save the results.

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title 2. Installation of Libraries
#@markdown StyleGAN2-ADA will be installed on your Google Drive to speed up the training process.

#@markdown Run this cell. If you have already installed the repository, it will skip the installation process and update the repository directory. If you haven't installed it, it will install all the necessary files.
#@markdown You may notice some errors: Ignore them for now, they are compatibility issues that do not affect our work.
import os
if os.path.isdir("/content/drive/MyDrive/colab-sg2-ada-pytorch"):
    %cd "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
elif os.path.isdir("/content/drive/"):
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir colab-sg2-ada-pytorch
    %cd colab-sg2-ada-pytorch
    !git clone https://github.com/angelv-salazar/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU -O /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/wikiart.pkl
else:
    !git clone https://github.com/angelv-salazar/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    %cd pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU
    %cd ../

!pip uninstall jax jaxlib -y
!pip install "jax[cuda11_cudnn805]==0.3.10" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install torch==1.8.1 torchvision==0.9.1
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git
!pip install ninja

%cd "/content/drive/My Drive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
!git config --global user.name "test"
!git config --global user.email "test@test.com"
!git fetch origin
!git pull
!git stash
!git checkout origin/main -- train.py generate.py legacy.py closed_form_factorization.py flesh_digression.py apply_factor.py README.md calc_metrics.py training/stylegan2_multi.py training/training_loop.py util/utilgan.py

In [ ]:
#@title 3. Compress our dataset into a zip file
#@markdown Write the directory of your Dataset.

#@markdown **REMEMBER:** The image path must not contain spaces " ", to avoid errors use a symbol instead "_"
input_dir = '/content/drive/MyDrive/data/gan01/images' #@param {type: "string"}
#@markdown Path to the *zip* file where the converted *Dataset* will be stored. You must create it the first time.
dataset_file = '/content/drive/MyDrive/data/gan01/verify.zip' #@param {type: "string"}

if not dataset_file.endswith('.zip'):
  dataset_file += '.zip'

!python dataset_tool.py --source {input_dir} --dest {dataset_file}

In [ ]:
#@title 4. Custom Model Training

#@markdown Enter the path to the *zip* file of the *Dataset*
dataset = "/content/drive/MyDrive/data/gan01/verify.zip" #@param {type: "string"}

#@markdown For transfer learning, set it to **ffhq256**, **ffhq512** or **ffhq1024** according to the resolution of your images.
#@markdown If you want to resume the training process, please provide the path to your latest *.pkl* file
resume_from = "ffhq512" #@param {type: "string"}

#don't edit this unless you know what you're doing :)
!python train.py --outdir ./results --snap=1 --cfg='11gb-gpu' --data={dataset} --aug=noaug --mirror=False --mirrory=False --metrics=None --resume={resume_from}

### While you are training...
Once the cell above is running, it should be training!

Don't close this tab! Colab must be open and operational to continue training.

Every 40 minutes or so a new line should be added to your output, indicating that you are still training. Depending on your snapshot_count settings, you should see the results folder (/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results) in your Google drive folder filling up with both samples (fakesXXXXXx.jpg) and model weights (model weights) (network-snapshot-XXXXXX.pkl). It's worth looking at samples while training, but don't worry too much about each individual sample.

Once Colab is powered off, you can plug it back into the housing and rerun each cell from top to bottom. Make sure to update the resume_from path to continue training from the latest model.